# Variables son almacenadas tanto en filas como en columnas

## "Housekeeping"

In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.set_option("display.max_columns", 40)

## Ejemplo: Clima

El [Global Historical Climatology Network](https://www.ncdc.noaa.gov/data-access/land-based-station-data/land-based-datasets/global-historical-climatology-network-ghcn) recolecta datos de clima de manera diaria. Para este ejemplo, se usan datos de una estación climática en Mexico (MX17004).

### Cargando datos

El raw dataset viene en un formato que es una mezcla de datos de ancho fijo con uso ocasional de caracteres como separadores. Se necesita hacer una limpieza al respecto.

In [3]:
# Extract the data as one column and
# use string slicing to obtain groups of columns.
weather = pd.read_csv("data/weather.txt", header=None, sep="^")

# First, remove the weird character separators,
# then split the columns by whitespace, and
# finally name them appropriately.
days = (
    weather[0]
    .map(lambda x: x[21:])
    .str.replace("OI", "  ")
    .str.replace("OS", "  ")
    .str.replace("SI", "  ")
    .str.replace("I", " ")
    .str.replace("S", " ")
    .str.replace("B", " ")
    .str.replace("D", " ")
    .map(str.lstrip)
    .str.split(r"\s+", expand=True)
)[list(range(31))].rename(columns={i: f"d{i+1}" for i in range(31)})

# The non-temperature columns can be extracted as simple slices.
weather = pd.DataFrame(
    data={
        "id": weather[0].map(lambda x: x[:11]),
        "year": weather[0].map(lambda x: x[11:15]).astype(int),
        "month": weather[0].map(lambda x: x[15:17]).astype(int),
        "element": weather[0].map(lambda x: x[17:21]).str.lower(),
    }
)

# The temperatures were stored as whole integers
# with -9999 indicating missing values.
for i in range(1, 32):
    weather[f"d{i}"] = days[f"d{i}"].astype(float) / 10
weather = weather.replace(-999.9, np.NaN)

# Discard the non-temperature observations and
# sort the dataset as in the paper.
weather = (
    weather[weather["element"].isin(["tmax", "tmin"])]
    .sort_values(["id", "year", "month", "element"])
    .reset_index(drop=True)
)

### Messy Data

A continuación hay un dataset que asumimos que ha sido proporcionado "crudo" como está al analista de datos, es decir, el analista no realizó el trabajo de "parsing" anterior, sino un tercero anteriormente.

> La forma más complicada de *messy data* ocurre cuando las variables se almacenan tanto en filas como en columnas. La Tabla 11 muestra datos meteorológicos diarios del Global Historical Climatology Network para una estación meteorológica (MX17004) en México durante cinco meses en 2010. Tiene variables en
columnas individuales (`"id"`, `"year"`, `"month"`), distribuidas en columnas (`day`, `"d1"`–`"d31"`) y en filas (`"tmin"` y `"tmax"` para las temperaturas mínima y máxima). Los meses con menos de 31 días tienen valores faltantes para los últimos días del mes. La columna `"element"` no es una variable: almacena los *nombres* de las variables.

In [4]:
weather[(weather["year"] == 2010)].head(10)

,id,year,month,element,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d20,d21,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
1099,MX000017004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.8,NaN
1100,MX000017004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.5,NaN
1101,MX000017004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1102,MX000017004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1103,MX000017004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,NaN,NaN,NaN,34.5,NaN,NaN,NaN,NaN,NaN,31.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1104,MX000017004,2010,3,tmin,NaN,NaN,NaN,NaN,14.2,NaN,NaN,NaN,NaN,16.8,NaN,NaN,NaN,NaN,NaN,17.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1105,MX000017004,2010,4,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.3,NaN,NaN,NaN,NaN
1106,MX000017004,2010,4,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.7,NaN,NaN,NaN,NaN
1107,MX000017004,2010,5,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.2,NaN,NaN,NaN,NaN
1108,MX000017004,2010,5,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.2,NaN,NaN,NaN,NaN


### Molten Data

> Para hacer este dataset tidy, primero le hacemos un `melt`  con colvars `"id"`, `"year"`, `"month"`, y la columna que contiene los nombres de las variables reales, `"element"` [...]. Para la presentación, hemos descartado los valores faltantes, haciéndolos implícitos en lugar de explícitos. Esto es permisible porque sabemos cuántos días hay en cada mes y podemos reconstruir fácilmente los valores faltantes explícitos.

In [5]:
# Melt the dataset and extract a date column.
molten_weather = (
    pd.melt(weather, id_vars=["id", "year", "month", "element"], var_name="day")
    .assign(day=lambda x: x["day"].str.extract("(\d+)").astype(int))
    .assign(date=lambda x: pd.to_datetime(x[["year", "month", "day"]], errors="coerce"))
)
molten_weather = molten_weather[["id", "date", "element", "value"]]

# Make the missing values implicit.
molten_weather = molten_weather[molten_weather["value"].notnull()]

# Sort the data as in the paper.
molten_weather = molten_weather.sort_values(["id", "date", "element"])
molten_weather = molten_weather.reset_index(drop=True)

> Este conjunto de datos está mayormente tidy, pero tenemos dos variables almacenadas en filas: `"tmin"` y `"tmax"`, el tipo de observación.

In [6]:
molten_weather[(molten_weather["date"].dt.year == 2010)].head(10)

,id,date,element,value
23183,MX000017004,2010-01-30,tmax,27.8
23184,MX000017004,2010-01-30,tmin,14.5
23185,MX000017004,2010-02-02,tmax,27.3
23186,MX000017004,2010-02-02,tmin,14.4
23187,MX000017004,2010-02-03,tmax,24.1
23188,MX000017004,2010-02-03,tmin,14.4
23189,MX000017004,2010-02-11,tmax,29.7
23190,MX000017004,2010-02-11,tmin,13.4
23191,MX000017004,2010-02-23,tmax,29.9
23192,MX000017004,2010-02-23,tmin,10.7


### Tidy Data

> Arreglar esto requiere la operación `cast`(conversión) o `unstack`(desapilar). Esto realiza el inverso del "fundido" (melting), rotando la variable `element` de vuelta hacia las columnas.

A continuación, [pd.DataFrame.unstack()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.unstack.html) usa un DataFrame's index como columnas para poder desapilar.


In [7]:
tidy_weather = molten_weather.set_index(["id", "date", "element"]).unstack()

# Make the column headers look as in the paper.
tidy_weather.columns = tidy_weather.columns.droplevel(0)
tidy_weather.columns.name = None
tidy_weather = tidy_weather.reset_index()

> Esta forma está tidy. Hay una variable en cada columna, y cada fila representa la observación de un día.

In [8]:
tidy_weather[(tidy_weather["date"].dt.year == 2010)].head(10)

,id,date,tmax,tmin
12087,MX000017004,2010-01-30,27.8,14.5
12088,MX000017004,2010-02-02,27.3,14.4
12089,MX000017004,2010-02-03,24.1,14.4
12090,MX000017004,2010-02-11,29.7,13.4
12091,MX000017004,2010-02-23,29.9,10.7
12092,MX000017004,2010-03-05,32.1,14.2
12093,MX000017004,2010-03-10,34.5,16.8
12094,MX000017004,2010-03-16,31.1,17.6
12095,MX000017004,2010-04-27,36.3,16.7
12096,MX000017004,2010-05-27,33.2,18.2
